## Import Packages

In [1]:
import numpy as np
import pandas as pd
import glob
import os

## Import CSVs

In [2]:
# Current Path
ROOT_DIR = os.path.abspath(os.curdir)
path = ROOT_DIR + '/financial_data'
print(path)

csvfiles = glob.glob(path + "/*.csv")
# Creating a dataframe for each CSV file
dfs = [pd.read_csv(file) for file in csvfiles]


C:\Users\Dell\Documents\B.Sc IT (Hons.) Computing and Business\FYP - Luke Bezzina\Code\preprocessingHistoricalData/financial_data


# New Dataframe

In [3]:
processed_df_coll = {}

## Pre-Process

In [4]:
prev_row = None
# Up to 0.2% deviation from close price from day before is considered Neutral movement
neutral_percentage = 0.002

for df in dfs:
    processed_df = pd.DataFrame(columns = ['Date', 'd10_pd', 'd9_pd', 'd8_pd', 'd7_pd', 'd6_pd',
                                           'd5_pd', 'd4_pd', 'd3_pd', 'd2_pd', 'd1_pd', 'Trend',
                                           'VolumeTrend', 'Volatility', 'PreviousClose', 'PriceDirection'])

    # Creating additional feature columns for each dataframe
    iterables = df.itertuples(index=True, name='Pandas')
    n = df.columns.get_loc('Name') # Name column in df
    df_name = df.iat[1, n] # Name of equity/etf

    # Iterating all rows in dataframe
    # Starting from size n + 1 - prices from past n days used per record
    for i in range(11, len(df) - 2 ):
        close = df.columns.get_loc('Close')
        volume = df.columns.get_loc('Volume')
        df_date = df.columns.get_loc('Date')

        # Price Trend Handling
        price_today = df.iat[i + 0, close]
        price_tomorrow = df.iat[i + 1, close]
        price_2days_after = df.iat[i + 2, close]

        close_price = (price_today + price_tomorrow + price_2days_after)/3
        prev_close_price = df.iat[i - 1, close]
        price_change = close_price - prev_close_price

        if price_change > (close_price * neutral_percentage):
            direction = 'Positive'
        elif price_change < 0 and abs(price_change) > (close_price * neutral_percentage):
            direction = 'Negative'
        else:
            direction = 'Neutral'

        # Price Difference Handling
        initial_close = df.iat[i - 11, close]

        d10_pd = df.iat[i - 10, close] - initial_close
        d9_pd = df.iat[i - 9, close] - initial_close
        d8_pd = df.iat[i - 8, close] - initial_close
        d7_pd = df.iat[i - 7, close] - initial_close
        d6_pd = df.iat[i - 6, close] - initial_close
        d5_pd = df.iat[i - 5, close] - initial_close
        d4_pd = df.iat[i - 4, close] - initial_close
        d3_pd = df.iat[i - 3, close] - initial_close
        d2_pd = df.iat[i - 2, close] - initial_close
        d1_pd = df.iat[i - 1, close] - initial_close

        # Volume Difference Handling
        initial_vol = df.iat[i - 11, volume] + 1 # Since volume traded can be 0, handling DIV by 0 issues

        vd10_pd = df.iat[i - 10, volume] / initial_vol
        vd9_pd = df.iat[i - 9, volume] / initial_vol
        vd8_pd = df.iat[i - 8, volume] / initial_vol
        vd7_pd = df.iat[i - 7, volume] / initial_vol
        vd6_pd = df.iat[i - 6, volume] / initial_vol
        vd5_pd = df.iat[i - 5, volume] / initial_vol
        vd4_pd = df.iat[i - 4, volume] / initial_vol
        vd3_pd = df.iat[i - 3, volume] / initial_vol
        vd2_pd = df.iat[i - 2, volume] / initial_vol
        vd1_pd = df.iat[i - 1, volume] / initial_vol

        date_ts = df.iat[i,df_date]

        # 10-day windows for close price and volume
        pds = [d1_pd, d2_pd, d3_pd, d4_pd, d5_pd, d6_pd, d7_pd, d8_pd, d9_pd, d10_pd]
        vpds = [vd1_pd, vd2_pd, vd3_pd, vd4_pd, vd5_pd, vd6_pd, vd7_pd, vd8_pd, vd9_pd, vd10_pd]

        # volatility of prices window
        volatility = np.std(pds)

        # finding trends of close and volume in terms of gradient (slope)
        slope, intercept = np.polyfit([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], pds, 1)
        slopeVol, interceptVol = np.polyfit([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], vpds, 1)

        new_row = {'Date':date_ts, 'd10_pd':d10_pd, 'd9_pd':d9_pd, 'd8_pd':d8_pd, 'd7_pd':d7_pd,
                   'd6_pd':d6_pd, 'd5_pd':d5_pd, 'd4_pd':d4_pd, 'd3_pd':d3_pd, 'd2_pd':d2_pd,
                   'd1_pd':d1_pd, 'Trend':slope, 'VolumeTrend':slopeVol, 'Volatility':volatility,
                   'PreviousClose':prev_close_price, 'PriceDirection':direction}

        #append row to the dataframe
        processed_df = processed_df.append(new_row, ignore_index=True)

    print(processed_df)
    processed_df_coll[df_name] = processed_df


           Date  d10_pd  d9_pd  d8_pd  d7_pd  d6_pd  d5_pd  d4_pd  d3_pd  \
0    2017-01-19    0.40  -0.41  -0.09   0.78   2.18   2.34   1.80   1.35   
1    2017-01-20   -0.81  -0.49   0.38   1.78   1.94   1.40   0.95   0.05   
2    2017-01-23    0.32   1.19   2.59   2.75   2.21   1.76   0.86   1.75   
3    2017-01-24    0.87   2.27   2.43   1.89   1.44   0.54   1.43   1.05   
4    2017-01-25    1.40   1.56   1.02   0.57  -0.33   0.56   0.18   0.92   
..          ...     ...    ...    ...    ...    ...    ...    ...    ...   
989  2020-12-22    0.42  -0.05   0.78  -0.16  -0.51  -0.20  -0.35  -0.41   
990  2020-12-23   -0.47   0.36  -0.58  -0.93  -0.62  -0.77  -0.83  -1.12   
991  2020-12-24    0.83  -0.11  -0.46  -0.15  -0.30  -0.36  -0.65  -1.06   
992  2020-12-28   -0.94  -1.29  -0.98  -1.13  -1.19  -1.48  -1.89  -2.51   
993  2020-12-29   -0.35  -0.04  -0.19  -0.25  -0.54  -0.95  -1.57  -1.16   

     d2_pd  d1_pd     Trend  VolumeTrend  Volatility  PreviousClose  \
0     0.45   1.3

# Data Export

In [5]:
for key, df in processed_df_coll.items():
    df.to_csv(ROOT_DIR+'\\classification_10d_3dprice\\'+key+"_3d.csv", index=False)

print("Export Complete!")

Export Complete!
